In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading Excel file
# bread = pd.read_excel('raw_bread.xlsx')
# bread = pd.read_csv('raw_bread.csv')
#

In [3]:
## Here we have transaction data, which include column, Date,Time,Transaction,Item
## we should remove duplicate transaction, it shows quantity of item in same transaction,
## it is not needed in appriori aglo as we only care about different item in particular transaction
# bread

In [4]:
## dropping Duplicate Transaction
# bread = bread.drop_duplicates()

In [5]:
## we need to split transaction data into Dataframe/tabular structure as follow
# new = bread['Date,Time,Transaction,Item'].str.split(',', n = 3, expand = True)

In [6]:
# import warnings
# warnings.filterwarnings('ignore')

In [7]:
## assigning column to data frame "bread"
# bread['Date'] = new[0]
# bread['Time'] = new[1]
# bread['Transaction'] = new[2]
# bread['Item'] = new[3]

In [8]:
# in this dataframe we only need column Trasaction and Item, rest is not needed in association mining rule
# bread[['Date', 'Time', 'Transaction', 'Item']].head(10)

In [9]:
# we need to convert cloumn transacton & item into Crosstab or we can say Binary Matrix as follow
# transaction = pd.crosstab(index= bread['Transaction'], columns= bread['Item'])
# transaction

In [10]:
## Just writing cdv file to check result
## I came to know that, we have one unwanted column named "NONE", we should remove it as follow and proceed further
#tab.to_csv('tab.csv')

In [11]:
## removing unwanted col "NONE"
# transaction = transaction.drop(['NONE'], axis = 1)
transaction = pd.read_csv('raw_bread_c.csv')

In [12]:
def APRIORI_MY(data, min_support=0.04,  max_length = 4):
    # Collecting Required Library
    import numpy as np
    import pandas as pd
    from itertools import combinations
    # Step 1:
    # Creating a dictionary to stored support of an itemset.
    support = {}
    L = list(data.columns)

    # Step 2:
    #generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        c = set(combinations(L,i))

    # Reset "L" for next ith iteration
        L =set()
    # Step 3:
        #iterate through each item in "c"
        for j in list(c):
            #print(j)
            sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
            if sup > min_support:
                #print(sup, j)
                support[j] = sup

                # Appending frequent itemset in list "L", already reset list "L"
                L = list(set(L) | set(j))

    # Step 4: data frame with cols "items", 'support'
    result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
    return(result)

In [13]:
## finding frequent itemset with min support = 4%
my_freq_itemset = APRIORI_MY(transaction, 0.04, 3)
my_freq_itemset.sort_values(by = 'Support', ascending = False)

,Items,Support
110095,"(Juice, Vegan Feast, Transaction)",4842.489875
105214,"(Adjustment, Afternoon with the baker, Transac...",4842.489875
6035,"(Pick and Mix Bowls, Tea, Transaction)",4842.489875
60080,"(Alfajores, Lemon and coconut, Transaction)",4842.489875
89320,"(Duck egg, Mortimer, Transaction)",4842.489875
...,...,...
49953,"(Tshirt, NONE, Jam)",1.000000
49952,"(Bare Popcorn, Mighty Protein, Olum & polenta)",1.000000
49951,"(Juice, Bowl Nic Pitt, Coffee granules)",1.000000
49950,"(Polenta, Lemon and coconut, Basket)",1.000000


In [14]:
# Creating ASSOCIATION_RULE_MY function to generate itemset based on minimun threshold confidence.

def ASSOCIATION_RULE_MY(df, min_threshold=0.5):
    import pandas as pd
    from itertools import permutations

    # STEP 1:
    #creating required varaible
    support = pd.Series(df.Support.values, index=df.Items).to_dict()
    data = []
    L= df.Items.values

    # Step 2:
    #generating rule using permutation
    p = list(permutations(L, 2))

    # Iterating through each rule
    for i in p:

        # If LHS(Antecedent) of rule is subset of RHS then valid rule.
        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            #print(i, conf)
            if conf > min_threshold:
                #print(i, conf)
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])


    # STEP 3:
    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", "consequent support",
                                        "support", "confidence", "Lift", "Leverage", "Convection"])
    return(result)

In [ ]:
## Rule with minimun confidence = 50%
my_rule = ASSOCIATION_RULE_MY(my_freq_itemset, 0.5)
my_rule

In [ ]:
my_rule.sort_values(by='Lift', ascending= False).head(10)

In [ ]:
# Loading standard package
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
## finding frequent itemset with min support = 4%
frequent_itemset = apriori(df = transaction, min_support= 0.04, use_colnames= True)
frequent_itemset.sort_values(by = 'support', ascending = False)

In [ ]:
# Createing associate rule such that item brought with conditional probability(Confidence) more than 50% with corresponding item

In [ ]:
## Rule with minimun confidence = 50%
Rules = association_rules(frequent_itemset, min_threshold= 0.5)
Rules

In [ ]:
## Finally sorting results by Lift to get highly associated itemsets.
Rules.sort_values(by='lift', ascending= False).head(10)
